In [1]:
import yaml
import os
import pathlib
import json

In [2]:
refdict = yaml.load(open('BH76_ref_energies.yaml','r'), Loader=yaml.Loader)
kcal = 627.509
all_mols = {}

In [5]:
def process_files(input_dir):
    mad_energies = {}
    # Iterate through the directory and its subdirectories
    for root, dirs, files in os.walk(input_dir):
        for dir in dirs:
            mol_name = dir
            all_mols[mol_name] = True

        for file in files:
            if file == "struc.calc_info.json": 
                
                input_file_path = os.path.join(root, file)
                # pbe and pbescf differ in path (geometry folder..) need to adjust this
                mol_name = input_file_path.split('/')[6]

                f = open(input_file_path)
                data = json.load(f)
                #print(mol_name, data['return_energy'])
                mad_energies[mol_name] = data['return_energy']
    
    return mad_energies

In [4]:
path = os.path.abspath(os.getcwd())
input_directory = path+'/pbe'  # Change this to your desired input directory
mad_pbehf = process_files(input_directory)

In [6]:
mad_pbescf = process_files(path+'/pbe_scf/geometries')

In [7]:
print(len(mad_pbehf.keys()))
print(len(mad_pbescf.keys()))

86
87


In [8]:
print('MISSING: ')
cmdict = yaml.load(open('BH76_chg_2s.yaml','r'), Loader=yaml.Loader)
for i in all_mols.keys():
    if i in mad_pbescf.keys():
        pass
    else:
        #print(i, "\t", cmdict[i])
        print(i)

MISSING: 
.ipynb_checkpoints


In [11]:
#print(refdict[1])

In [9]:
# only for debug use
h_pbe = -0.4999410227691473
h_hf  = -0.4999535118271135

In [10]:
# iterate over Refdict, stoich keys lookup in madness energies dict, calc bh according to stoich, scale to kcal
#
# !! code does not support restart from h single electron wf, pbe@hf for h does not work
# here, TEMPORARILY, use hf/pbe energy instead
pbe_bh76_mad = {}
mad_pbehf['h'] = h_hf
for bh in refdict.keys():
    e_bh = 0   
    for system in refdict[bh]['Stoich'].keys():
        stoich_fac = refdict[bh]['Stoich'][system]
        e_bh += stoich_fac * mad_pbehf[system]
        #print(e_bh)

    pbe_bh76_mad[bh] = e_bh * kcal
    #print(bh, pbe_bh76_mad[bh])

In [11]:
pbescf_bh76_mad = {}
for bh in refdict.keys():
    e_bh = 0   
    for system in refdict[bh]['Stoich'].keys():
        stoich_fac = refdict[bh]['Stoich'][system]
        e_bh += stoich_fac * mad_pbescf[system]
        #print(e_bh)

    pbescf_bh76_mad[bh] = e_bh * kcal
    #print(bh, pbe_bh76_mad[bh])

In [ ]:
print(37, pbe_bh76_mad[37], 48.10 - pbe_bh76_mad[37])

In [12]:
ref_bh, mad_pbehf_bh,mad_pbescf_bh = [], [], []
for i in refdict.keys():
    #print(i,'\t', "%.2f" % pbe_bh76_mad[i], '\t\t', refdict[i]['Ref'])
    mad_pbehf_bh.append(pbe_bh76_mad[i])
    mad_pbescf_bh.append(pbescf_bh76_mad[i])
    ref_bh.append(refdict[i]['Ref'])

In [13]:
error_bh = []
error_bhscf = []

for i in pbe_bh76_mad.keys():
    error_bh.append(pbe_bh76_mad[i] - refdict[i]['Ref'])
    error_bhscf.append(pbescf_bh76_mad[i] - refdict[i]['Ref'])
    #print(i, pbe_bh76_mad[i] - refdict[i]['Ref'])
#print(error)

In [14]:
from scipy import stats
print(stats.median_abs_deviation(error_bh))
print(stats.median_abs_deviation(error_bhscf))

2.5227178776254644
2.6936794610096655


In [15]:
from statistics import mean
print(mean(error_bh))
print(mean(error_bhscf))

-0.7150351323570919
-8.707904125274554


In [16]:
#for i in error_bhscf:
#    print(i)

-7.608414624509598
-29.76006267097823
-15.055559339075145
-15.055559339075145
-8.183158871218762
-8.183158871218762
-12.014601435748322
-15.749821831227898
-10.734808480831585
-24.977849536540347
-12.411076961188627
-18.361961637610385
-4.664620273798329
-4.664620273798329
-5.865548964325618
-5.865548964325618
-5.511654847975663
-5.511654847975663
-6.1343334773836045
-6.1343334773836045
-3.715658872980214
-7.267267230043016
-3.6319242972206562
-8.42457185276848
-4.050890775941575
-5.347449558812254
-6.281082910467021
-4.794412552913407
-9.251664987928908
-1.968285522076215
-4.806209779959597
1.76749927000332
-1.9640808643996712
-1.7128193375848824
-4.622739628949702
-3.074725055384345
-2.5700907843398255
-2.335287452213848
-5.422824844240358
-9.296642473073067
-11.300609842228301
-7.756576879092918
-7.936613716900597
-5.5039488215979535
-11.486747789551277
-10.275379721714078
-5.952912344013283
-5.952912344013283
-14.901661788114705
-14.139303808974033
-7.271117961998344
-8.71227069554